In [2]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [5]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")
SPY_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPY")
SPXT_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPXT")

portfolio = portfolio.set_index("Date")
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()
SPY_Idx = SPY_Idx.set_index("Date").sort_index()
SPXT_Idx = SPXT_Idx.set_index("Date").sort_index()

portfolio.loc[:,"TLTCumDiv"]=portfolio.loc[:,"TLT Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueTLT"]=portfolio.loc[:,"TLT Position"]+portfolio.loc[:,"TLTCumDiv"]
portfolio = portfolio.sort_index()

TLTCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueTLT"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
TLTCorrData.loc[:,"TotalReturnsTLT"] = TLTCorrData["TotalValueTLT"].pct_change()
TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(HYyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_SPREAD"})
TLTCorrData.loc[:,"HY_Index_OAS_Diff"]=TLTCorrData["HY_Index_OAS"].diff()
TLTCorrData.loc[:,"HY_INDEX_Z_SPREAD_Diff"]=TLTCorrData["HY_INDEX_Z_SPREAD"].diff()

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
TLTCorrData.loc[:,"10yYieldDiff"] = TLTCorrData["10y_UST_INDEX_PX"].diff()

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"IG_Index_OAS"})
TLTCorrData.loc[:,"IG_Index_OAS_Diff"]=TLTCorrData["IG_Index_OAS"].diff()
TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_SPREAD"})
TLTCorrData.loc[:,"IG_Index_Z_SPREAD_Diff"]=TLTCorrData["IG_INDEX_Z_SPREAD"].diff()

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(SPY_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"PX_LAST":"SPY_Idx_LAST_PX"})
TLTCorrData.loc[:,"ReturnSPY_Idx"] = TLTCorrData["SPY_Idx_LAST_PX"].pct_change()

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(SPXT_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"PX_LAST":"SPXT_Idx_LAST_PX"})
TLTCorrData.loc[:,"ReturnSPXT_Idx"] = TLTCorrData["SPXT_Idx_LAST_PX"].pct_change()

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

TLTCorrData = TLTCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
TLTCorrData = TLTCorrData.rename(columns={"PX_LAST":"Vix Idx"})

TLTCorrData=TLTCorrData.dropna()
TLTCorrData = TLTCorrData.sort_index(ascending=False)

In [9]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(TLTCorrData[["10yYieldDiff","MOVE Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["10yYieldDiff","MOVE Idx"]))

X_pca = pcav1.transform(X_Std)

TLTCorrData.loc[:,"PC1"]=X_pca[:,0]
TLTCorrData.loc[:,"PC2"]=X_pca[:,1]
#TLTCorrData.loc[:,"PC3"]=X_pca[:,2]
#TLTCorrData.loc[:,"PC4"]=X_pca[:,3]
#TLTCorrData.loc[:,"PC5"]=X_pca[:,4]

X = sm.add_constant(TLTCorrData[["PC1","PC2","PC3"]])
Y = TLTCorrData["TotalReturnsTLT"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.50977854 0.49022146]
   10yYieldDiff  MOVE Idx
0      0.707107  0.707107
1      0.707107 -0.707107


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsTLT   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.2219
Date:                Thu, 15 Jan 2026   Prob (F-statistic):              0.881
Time:                        20:32:34   Log-Likelihood:                 4005.1
No. Observations:                1240   AIC:                            -8002.
Df Residuals:                    1236   BIC:                            -7982.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003      0.000     -0.964      0.335      -0.001       0.000
PC1           -0.0001      0.000     -0.313      0.754      -0.001       0.001
PC2           -0.0001      0.000     -0.352      0.725      -0.001       0.000
PC3            0.0003      0.000      0.785      0.433      -0.000       0.001
==============================================================================
Omnibus:                       10.488   Durbin-Watson:                   2.102
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               15.072
Skew:                           0.033   Prob(JB):                     0.000534
Kurtosis:                       3.536   Cond. No.                         2.18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""